In [ ]:
options(jupyter.rich_display = F)

# WRANGLING AN ECONOMIC DATA SET: IMF WORLD ECONOMIC OUTLOOK, CONTINUED

**by Serhat Çevikel**

We wrangle and analyze 2016 data of World Economic Outlook dataset by IMF

First please download following two data files:

[weo_2016_wide_2.csv](~/file/weo_2016_wide_2.csv)

[weo_description.csv](~/file/weo_description.csv)

And read the data into R as such:

In [ ]:
weo_data <- read.csv("~/file/weo_2016_wide_2.csv")
weo_desc <- read.csv("~/file/weo_description.csv")

Let's take a quick snapshot of the data:

In [ ]:
str(weo_data)

In [ ]:
str(weo_desc)

In [ ]:
weo_desc

There are 45 numeric variables for 194 countries (some of the data might be missing). We will be interested in only a few of those series

## ADDING SAVINGS GAP

- NID_NGDP is "Total investment"
- NGSD_NGDP is "Gross national savings"

Now let's add a new variable "savings_gap":

In [ ]:
weo_data$savings_gap <- with(weo_data, NGSD_NGDP - NID_NGDP)

## DISCRETIZE INCOME VARIABLE

PPPPC is Gross domestic product per capita, current prices with	purchasing power parity and in international dollars

Now we will three categories of income level: Low, medium and high

In [ ]:
weo_data$income_level <- with(weo_data, cut(PPPPC,
              breaks = c(0, 5000, 20000,
                         max(PPPPC, na.rm = T)),
              labels = c("low", "medium", "high")))

Let's see in a single plot:
- the distribution across levels
- and the dispersion of income within income levels
- scatterplot across savings gap and incomee

In [ ]:
options(repr.plot.width=15, repr.plot.height=15) # enlarge the following plots, this is jupyter specific

In [ ]:
par(mfrow = c(2,2))
barplot(table(weo_data$income_level))
with(weo_data, boxplot(PPPPC ~ income_level))
with(weo_data, plot(savings_gap, PPPPC, col = income_level))

See that high income level has more outliers

## SUMMARIZE DATA

Let's get the median of all variables across income categories

In [ ]:
weo_data_subset1 <- weo_data[,setdiff(names(weo_data),
                                             c("WEO.Country.Code",
                                               "ISO",
                                               "Country",
                                              "income_level"))]

weo_sum <- aggregate(weo_data_subset1,
          by = weo_data["income_level"],
          FUN = median,
          na.rm = T)

In [ ]:
weo_sum

Now let's reshape this data frame two times and merge with descriptions so that it is in a more interpretable format

First melt it:

In [ ]:
cols <- names(weo_sum)[-1]

weo_long <- reshape(weo_sum,
                      idvar = c("income_level"),
                      varying = cols,
                        times = cols,
                    timevar = "variable",
                      v.name = "value",
                      direction = "long")

rownames(weo_long) <- NULL

In [ ]:
weo_long

And then cast it:

In [ ]:
weo_wide <- reshape(weo_long,
                      idvar = c("variable"),
                      v.names = "value",
                      timevar = "income_level",
                      direction = "wide")

weo_wide

And finally merge it:

In [ ]:
weo_sum_merged <- merge(
    weo_desc[c("WEO.Subject.Code", "Subject.Descriptor", "Units")],
    weo_wide,
    by.y = "variable",
    by.x= "WEO.Subject.Code",
    all.y = T)

weo_sum_merged

Take some time to interpret this data frame

## SCATTERPLOTS

Create scatterplots of selected variables:

In [ ]:
cols2 <- c("NGDP_RPCH", "GGXONLB_NGDP", "GGXWDN_NGDP", "BCA_NGDPD", "savings_gap")
newnames <- c("GDP_growth", "Primary_balance", "Net_debt", "Current_account", "Saving_gap")

weo_data_subset2 <- weo_data[cols2]
names(weo_data_subset2) <- newnames

plot(weo_data_subset2, col = weo_data$income_level)

We see Saving Gap is nearly identical to Current Account Balance for many countries

## CORRELATIONS

We may look at correlations:

In [ ]:
weo_desc

We exclude variables of National Currency or scales of nominal quantities like million or billions

In [ ]:
vars3 <- with(weo_desc, WEO.Subject.Code[which(Units != "National currency" & !(Scale %in% c("Millions", "Billions")))])
vars3 <- as.character(vars3)
length(vars3)

See the left variables' descriptions:

In [ ]:
with(weo_desc, weo_desc[WEO.Subject.Code %in% vars3,])

Subset the data set for only those variables:

In [ ]:
weo_data_subset3 <- weo_data[,vars3] 
weo_data_subset3

Create the correlation matrix of all variables and round the results:

In [ ]:
cormat <- round(cor(weo_data_subset3, use = "pairwise.complete.obs"), 2)
cormat

We leave the upper triangle values by setting diagonal and lower triangle to NA, to prevent duplicate values from the symmetric nature:

In [ ]:
cormat[row(cormat) >= col(cormat)] <- NA

In [ ]:
cormat

Order the descriptions according to the correlation matrix (so that we can match them later):

In [ ]:
desc_ordered <- weo_desc[match(rownames(cormat), weo_desc$WEO.Subject.Code),]
desc_ordered

An combine descriptor and units into a single column, so that we see all descriptor information in a single column:

In [ ]:
desc_ordered$longdesc <- with(desc_ordered, paste(Subject.Descriptor, Units, sep = " - "))

desc_ordered

Get the row and column indices of sufficiently high (but less than perfect) correlations:

In [ ]:
highcor <- which(abs(cormat) > 0.5 & abs(cormat) < 0.8, arr.ind = T)
rownames(highcor) <- NULL
highcor

And see which variable pairs are highly correlated:

In [ ]:
pairs <- as.data.frame(t(apply(highcor, 1, function(x) desc_ordered$longdesc[x])))

And add the correlations:

In [ ]:
pairs$cor <- cormat[highcor]

In [ ]:
pairs[order(-abs(pairs$cor)),]

Now some definitions:

> The output gap is actual minus potential output, as a percentage of potential output. Structural balances are expressed as a percentage of potential output. The structural balance is the actual net lending/borrowing minus the effects of cyclical output from potential output, corrected for one-time and other factors, such as asset and commodity prices and output composition effects. Changes in the structural balance consequently include effects of temporary fiscal measures, the impact of fluctuations in interest rates and debt-service costs, and other noncyclical fluctuations in net lending/borrowing. The computations of structural balances are based on the IMF staff’s estimates of potential GDP and revenue and expenditure elasticities. (See Annex I of the October 1993 WEO.) Net debt is calculated as gross debt minus financial assets corresponding to debt instruments. Estimates of the output gap and of the structural balance are subject to significant margins of uncertainty.

(https://www.elibrary.imf.org/view/IMF081/28248-9781513508214/28248-9781513508214/ch04.xml?redirect=true)

> The output gap is an economic measure of the difference between the actual output of an economy and its potential output. Potential output is the maximum amount of goods and services an economy can turn out when it is most efficient—that is, at full capacity. Often, potential output is referred to as the production capacity of the economy. ...

> Various methodologies are used to estimate potential output, but they all assume that output can be divided into a trend and a cyclical component. The trend is interpreted as a measure of the economy’s potential output and the cycle as a measure of the output gap. The trick to estimating potential output, therefore, is to estimate trends—that is, to remove the cyclical changes. ...

> A common method of measuring potential output is the application of statistical techniques that differentiate between the short-term ups and downs and the long-term trend. The Hodrick-Prescott filter is one popular technique for separating the short from the long term. Other methods estimate the production function, a mathematical equation that calculates output based on an economy’s inputs, such as labor and capital. Trends are estimated by removing the cyclical changes in the inputs.

(https://www.imf.org/external/pubs/ft/fandd/2013/09/basics.htm)

Now let's create the scatterplots of just those variable pairs into a single grid:

In [ ]:
nrow(highcor)

In row major order with mfrow:

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)
highcor3 <- cbind(highcor, seq_along(highcor[,1]))
highcor3

In [ ]:
par(mfrow = c(4, 2))
apply(highcor3, 1, function(x) plot(weo_data_subset3[,x[-3]],
                                    xlab = desc_ordered$longdesc[x[1]],
                                    ylab = strwrap(desc_ordered$longdesc[x[2]], width=35, simplify=FALSE),
                                    main = paste("Plot:", x[3]),
                                    col = weo_data$income_level,
                                    cex.lab = 1.5))

## REGRESSION MODELING

Now we will conduct a multiple linear regression analysis to understand the relationship among variables

Let's try to explain:

- NGDP_RPCH,"Gross domestic product, constant prices",Percent change,

with

- PPPSH,Gross domestic product based on purchasing-power-parity (PPP) share of world total,Percent,
- NID_NGDP,Total investment,Percent of GDP,
- PCPIPCH,"Inflation, average consumer prices",Percent change,
- GGXONLB_NGDP,General government primary net lending/borrowing,Percent of GDP,
- GGXWDN_NGDP,General government net debt,Percent of GDP

Let's user weo_data_subset3

In [ ]:
independent_vars <- c("PPPSH", "NID_NGDP", "PCPIPCH", "GGXONLB_NGDP", "GGXWDN_NGDP")
dependent_var <- "NGDP_RPCH"

In [ ]:
names(weo_data_subset3)

Let's first subset necessary variables 

In [ ]:
weo_data_subset4 <- weo_data_subset3[, c(dependent_var, independent_vars)]

And delete rows with missing values

In [ ]:
weo_data_subset5 <- na.omit(weo_data_subset4)

In [ ]:
str(weo_data_subset5)

Create the model formula object:

In [ ]:
modelx <- as.formula(paste(dependent_var, paste(independent_vars, collapse = "+"), sep ="~"))
modelx

Run the initial regression model:

In [ ]:
reg_mod <- lm(modelx, weo_data_subset5)

In [ ]:
summary(reg_mod)

As we see only NID_NGDP (Total investment,Percent of GDP) is a significant dependent variable for explaining the variation in NGDP_RPCH,"Gross domestic product, constant prices",Percent change

The coefficient is 0.156906.

So every unit increase in investment/GDP ratio results in 0.15% increase in GDP growth

Let's leave alone this single significant variable in the second model:

In [ ]:
reg_mod2 <- lm(NGDP_RPCH ~ NID_NGDP, weo_data_subset5)

In [ ]:
summary(reg_mod2)

Investment/GDP ratio alone explains around 20% cross-section variation in GDP growth rates across countries 

Plot the independent and dependent variables with the best fit line from the model:

In [ ]:
options(repr.plot.width=8, repr.plot.height=8) # shrink the following plots, this is jupyter specific

In [ ]:
plot(weo_data_subset5$NID_NGDP, weo_data_subset5$NGDP_RPCH)
abline(reg_mod2, col = "red")

Calculate the fitted values from the model:

In [ ]:
fitted2 <- predict(reg_mod2, weo_data_subset5)

And plot actual vs fitted values:

In [ ]:
plot(x = weo_data_subset5$NGDP_RPCH, y = fitted2,
xlab = "Actual",
ylab ="Fitted")
abline(0, 1, col = "red")

There are some outliers, let's take them out:

In [ ]:
weo_data_subset6 <- with(weo_data_subset5, weo_data_subset5[NGDP_RPCH >= quantile(NGDP_RPCH, 0.01) & NGDP_RPCH <= quantile(NGDP_RPCH, 0.99),])

And let's try to fit a quadratic model with uncorrelated polynomial terms using the poly() function:

In [ ]:
reg_mod3 <- lm(NGDP_RPCH ~ poly(NID_NGDP, 2), weo_data_subset6)

In [ ]:
summary(reg_mod3)

Calculate fitted values:

In [ ]:
fitted3 <- predict(reg_mod3, weo_data_subset6)

Plot the actual and fitted values:

In [ ]:
plot(x = weo_data_subset6$NGDP_RPCH, y = fitted3,
xlab = "Actual",
ylab ="Fitted")
abline(0, 1, col = "red")

It seems that the previous and better fit performance was a result of 2 influential observations (outliers) in the data

Modelling is an iterative, intuitive and creative process that you try to refine the model by enhancing variable selection, feature engineering with variable modification, model specification and case filtering against outliers

So the story of our model is not finished here!

You may continue to enhance it ...